In [187]:
from trueskill import Rating, rate
import pandas as pd

data = pd.read_csv("../data/results.csv")

In [211]:
rankings = {}
general_rankings = {}
conditions = data["conditions"].unique()
for condition in conditions:
    print("Condition: " + condition)
    rankings[condition] = {}
    condition_data = data[data["conditions"] == condition]
    events = condition_data["event_name"].unique()
    for event in events:
        event_data = condition_data[condition_data["event_name"] == event]
        event_ranks = [finish for finish in event_data["final_finish"]]
        event_rankings = []
        gen_event_rankings = []
        for driver, codriver, car, tyre, finish in zip(event_data["driver"], event_data["codriver"], event_data["car"], event_data["tyre"], event_data["final_finish"]):
            # event_rankings.append({driver: rankings[condition].get(driver, Rating()), codriver: rankings[condition].get(codriver, Rating()), car: rankings[condition].get(car, Rating()), tyre: rankings[condition].get(tyre, Rating())})
            gen_event_rankings.append({driver: general_rankings.get(driver, Rating()), codriver: general_rankings.get(codriver, Rating()), car: general_rankings.get(car, Rating()), tyre: general_rankings.get(tyre, Rating())})
            # event_rankings.append({driver: rankings[condition].get(driver, Rating())})
            event_rankings.append({driver: rankings[condition].get(driver, Rating()), codriver: rankings[condition].get(codriver, Rating()), car: rankings[condition].get(car, Rating())})
        newRate = rate(event_rankings, ranks=event_ranks) #, weights=[(1, 0.2, 0.2, 0.2) for _ in range(len(event_rankings)])
        genNewRate = rate(gen_event_rankings, ranks=event_ranks)
        # print(event_rankings)
        # print(event_ranks)
        # print(newRate)
        for ranking, gen_ranking in zip(newRate, genNewRate):
            rankings[condition].update(ranking)
            general_rankings.update(gen_ranking)

import json
print(json.dumps(general_rankings, indent=4, default=str))

Condition: asphalt-snow
Condition: snow-ice
Condition: gravel
Condition: asphalt
Condition: asphalt-gravel
Condition: gravel-asphalt
{
    "M\u00e4kinen Tommi": "trueskill.Rating(mu=60.643, sigma=1.993)",
    "Mannisenm\u00e4ki Risto": "trueskill.Rating(mu=53.851, sigma=2.861)",
    "Mitsubishi Lancer Evo VI": "trueskill.Rating(mu=-3.192, sigma=1.437)",
    "https://www.ewrc-results.com/images/tyres/michelin_red.png": "trueskill.Rating(mu=-120.602, sigma=1.010)",
    "Sainz Carlos": "trueskill.Rating(mu=52.652, sigma=1.405)",
    "Moya Luis": "trueskill.Rating(mu=45.222, sigma=1.976)",
    "Ford Focus WRC '00": "trueskill.Rating(mu=28.275, sigma=2.833)",
    "Kankkunen Juha": "trueskill.Rating(mu=42.121, sigma=3.086)",
    "Repo Juha": "trueskill.Rating(mu=41.317, sigma=3.001)",
    "Subaru Impreza S5 WRC '99": "trueskill.Rating(mu=31.634, sigma=2.816)",
    "https://www.ewrc-results.com/images/tyres/pirelli_red.png": "trueskill.Rating(mu=-112.574, sigma=0.999)",
    "Gardemeister Toni

In [212]:
# weightings = []
# for driver in range(0, 3):
#     for codriver in range(0, 3):
#         for car in range(0, 3):
#             for tyre in range(0, 3):
#                 weightings.append([driver, codriver, car, tyre])
# print(len(weightings))

weightings = []
for driver in range(0, 3):
    for codriver in range(0, 3):
        for car in range(0, 3):
            weightings.append([driver, codriver, car])
print(len(weightings))

group_weightings = {"RC1": 1, "RC2": 2, "RC3": 3, "RC4": 4, "RC5": 5}

27


In [213]:
from sklearn.metrics import mean_squared_error, r2_score
import json

def rmse_driver_order(true_order, predicted_order):
    true_order_dict = {driver: i for i, driver in enumerate(true_order)}
    predicted_order_dict = {driver: i for i, driver in enumerate(predicted_order)}
    true_order_indexed = [true_order_dict[driver] for driver in predicted_order_dict.keys()]
    predicted_order_indexed = [predicted_order_dict[driver] for driver in predicted_order_dict.keys()]
    rmse = mean_squared_error(true_order_indexed, predicted_order_indexed, squared=False)
    # print(true_order_dict, predicted_order_dict, true_order_indexed, predicted_order_indexed, rmse)
    return rmse

def r2_driver_order(true_order, predicted_order):
    true_order_dict = {driver: i for i, driver in enumerate(true_order)}
    predicted_order_dict = {driver: i for i, driver in enumerate(predicted_order)}
    true_order_indexed = [true_order_dict[driver] for driver in predicted_order_dict.keys()]
    predicted_order_indexed = [predicted_order_dict[driver] for driver in predicted_order_dict.keys()]
    r2 = r2_score(true_order_indexed, predicted_order_indexed)
    # print(true_order_dict, predicted_order_dict, true_order_indexed, predicted_order_indexed, rmse)
    return r2

In [206]:
condition_rmses = {}
for condition in conditions:
    print("CONDITION: " + condition.upper())
    condition_data = data[data["conditions"] == condition]
    weightings_rmses = {}
    for weighting in weightings:
        print(weighting)
        if sum(weighting) == 0:
            continue
        rmses = {}
        for event in condition_data["event_name"].unique():
            event_data = condition_data[condition_data["event_name"] == event]
            true_order = [driver["driver"] for _, driver in event_data.iterrows()]
            condition = event_data["conditions"].iloc[0]
            expected_finish_order = {}
            for _, entry in  event_data.sample(frac=1).iterrows():
                driver_rating = rankings[condition].get(entry["driver"], Rating())
                codriver_rating = rankings[condition].get(entry["codriver"], Rating())
                car_rating = rankings[condition].get(entry["car"], Rating())
                tyre_rating = rankings[condition].get(entry["tyre"], Rating())
                # rating_list = [driver_rating, codriver_rating, car_rating, tyre_rating]
                rating_list = [driver_rating, codriver_rating, car_rating]

                gen_driver_rating = general_rankings.get(entry["driver"], Rating())
                gen_codriver_rating = general_rankings.get(entry["codriver"], Rating())
                gen_car_rating = general_rankings.get(entry["car"], Rating())
                gen_tyre_rating = general_rankings.get(entry["tyre"], Rating())
                gen_rating_list = [gen_driver_rating, gen_codriver_rating, gen_car_rating, gen_tyre_rating]
                
                # weights = [4, 1, 0, 0]
                weights = weighting
                combined_rating = sum([rating_list[i].mu * weights[i] for i in range(len(rating_list))]) / sum(weights) / group_weightings.get(entry["group"], 1)
                # gen_combined_rating = sum([gen_rating_list[i].mu * weights[i] for i in range(len(gen_rating_list))]) / sum(weights)

                # expected_finish_order[entry["driver"]] = combined_rating
                # expected_finish_order[entry["driver"]] = gen_combined_rating
                expected_finish_order[entry["driver"]] = driver_rating.mu

            sorted_driver_entries = sorted(expected_finish_order.items(), key=lambda x: x[1], reverse=True)
            predicted_order = [driver_entry[0] for driver_entry in sorted_driver_entries]
            # print(true_order, predicted_order)
            rmse = rmse_driver_order(true_order, predicted_order)
            rmses[event] = round(rmse, 2)
        print(sum(rmses.values())/len(rmses))
        weightings_rmses[str(weighting)] = (sum(rmses.values())/len(rmses))

    # print(json.dumps(weightings_rmses, indent=4, default=str))
    sorted_ranks = (sorted(weightings_rmses.items(), key=lambda k: k[1], reverse=True))
    print(sorted_ranks[-1])
    # for i, s in enumerate(sorted_ranks):
    #     print(f"{i}. {s[0]} RMSE: {s[1]}")

    condition_rmses[condition] = sorted_ranks[-1][0]

CONDITION: A
[0, 0, 0]
[0, 0, 1]


ZeroDivisionError: division by zero

In [214]:
condition_rmses = {}
for condition in conditions:
    rmses = {}
    condition_data = data[data["conditions"] == condition]
    print(condition)
    for event in condition_data["event_name"].unique():
        event_data = condition_data[condition_data["event_name"] == event]
        true_order = [driver["driver"] for _, driver in event_data.iterrows()]
        condition = event_data["conditions"].iloc[0]
        expected_finish_order = {}
        for _, entry in event_data.sample(frac=1).iterrows():
            expected_finish_order[entry["driver"]] = rankings[condition].get(entry["driver"], Rating())
        sorted_driver_entries = sorted(expected_finish_order.items(), key=lambda x: x[1], reverse=True)
        predicted_order = [driver_entry[0] for driver_entry in sorted_driver_entries]
        rmse = rmse_driver_order(true_order, predicted_order)
        rmses[event] = round(rmse, 2)
    condition_rmses[condition] = (sum(rmses.values())/len(rmses))
print(condition_rmses)


asphalt-snow
snow-ice
gravel
asphalt
asphalt-gravel
gravel-asphalt
{'asphalt-snow': 6.188571428571428, 'snow-ice': 7.076000000000001, 'gravel': 7.178994708994706, 'asphalt': 8.566935483870965, 'asphalt-gravel': 7.083333333333333, 'gravel-asphalt': 0.0}


In [ ]:
weighting = [4, 4, 2, 3]
# weighting = [5, 1, 1, 1]

In [205]:
event = data[data["event_name"] == "66. Rally Sweden 2018"]
condition = event["conditions"].iloc[0]
# condition = "asphalt"
weighting = json.loads(condition_rmses[condition])
print(condition, weighting)
event_rankings = {}
true_order = [driver["driver"] for _, driver in event.iterrows()]
for _, entry in event.sample(frac=1).iterrows():
    driver_rating = rankings[condition].get(entry["driver"], Rating())
    codriver_rating = rankings[condition].get(entry["codriver"], Rating())
    car_rating = rankings[condition].get(entry["car"], Rating())
    tyre_rating = rankings[condition].get(entry["tyre"], Rating())
    # rating_list = [driver_rating, codriver_rating, car_rating, tyre_rating]
    rating_list = [driver_rating, codriver_rating, car_rating]
    event_rankings[entry["driver"]] = sum([rating_list[i].mu * weighting[i] for i in range(len(rating_list))]) / sum(weighting)  / group_weightings.get(entry["group"], 1)
    # event_rankings[entry["driver"]] = (driver_rating.mu + codriver_rating.mu + car_rating.mu + tyre_rating.mu) / 4
    # event_rankings[entry["driver"]] = driver_rating
    # event_rankings[entry["driver"]] = (driver_rating.mu + codriver_rating.mu) / 2
sorted_ranks = (sorted(event_rankings.items(), key=lambda k: k[1], reverse=True))
predicted_order = [driver_entry[0] for driver_entry in sorted_ranks]
for i, s in enumerate(sorted_ranks):
    print(f"{i+1}. {s[0]}: {s[1]}")
print(true_order)
print(predicted_order)
print(rmse_driver_order(true_order, predicted_order))
print(r2_driver_order(true_order, predicted_order))

snow-ice [2, 2, 1]
1. Tänak Ott: trueskill.Rating(mu=50.684, sigma=3.448)
2. Mikkelsen Andreas: trueskill.Rating(mu=49.660, sigma=3.223)
3. Katsuta Takamoto: trueskill.Rating(mu=49.564, sigma=4.300)
4. Neuville Thierry: trueskill.Rating(mu=48.613, sigma=3.159)
5. Evans Elfyn: trueskill.Rating(mu=48.562, sigma=3.557)
6. Lappi Esapekka: trueskill.Rating(mu=47.180, sigma=3.931)
7. Tuohino Janne: trueskill.Rating(mu=44.023, sigma=3.684)
8. Paddon Hayden: trueskill.Rating(mu=43.986, sigma=4.678)
9. Ogier Sébastien: trueskill.Rating(mu=42.031, sigma=2.956)
10. Østberg Mads: trueskill.Rating(mu=41.090, sigma=2.767)
11. Monelius Mattias: trueskill.Rating(mu=40.749, sigma=5.832)
12. Latvala Jari-Matti: trueskill.Rating(mu=40.194, sigma=3.128)
13. Breen Craig: trueskill.Rating(mu=39.840, sigma=3.304)
14. Solberg Henning: trueskill.Rating(mu=37.483, sigma=2.584)
15. Kristoffersson Johan: trueskill.Rating(mu=37.257, sigma=4.288)
16. Adielsson Mattias: trueskill.Rating(mu=37.029, sigma=6.941)
17. V

In [194]:
upcoming_event = pd.read_csv("../data/upcoming.csv")

In [201]:
conditions = upcoming_event["conditions"].iloc[0]
weighting = json.loads(condition_rmses[conditions])
print(conditions, weighting)
event_rankings = {}
weighting = [15, 2, 1]
for _, entry in upcoming_event.sample(frac=1).iterrows():
    driver_rating = rankings[conditions].get(entry["driver"], Rating())
    codriver_rating = rankings[conditions].get(entry["codriver"], Rating())
    car_rating = rankings[conditions].get(entry["car"], Rating())

    rating_list = [driver_rating, codriver_rating, car_rating]
    event_rankings[entry["driver"]] = sum([rating_list[i].mu * weighting[i] for i in range(len(rating_list))]) / sum(weighting) / group_weightings.get(entry["group"], 1)

sorted_ranks = (sorted(event_rankings.items(), key=lambda k: k[1], reverse=True))
predicted_order = [driver_entry[0] for driver_entry in sorted_ranks]
for i, s in enumerate(sorted_ranks):
    print(f"{i+1}. {s[0]}: {s[1]}")


asphalt [2, 2, 1]
1. Evans Elfyn: 48.59973168992813
2. Neuville Thierry: 46.8328687197628
3. Tänak Ott: 45.74062729333154
4. Ogier Sébastien: 44.54805128524765
5. Katsuta Takamoto: 42.844963794751266
6. Lappi Esapekka: 41.71979403909941
7. Rovanperä Kalle: 39.923645761876195
8. Breen Craig: 35.932574748020095
9. Loubet Pierre-Louis: 35.80869330626374
10. Gryazin Nikolay: 25.54881517482831
11. Lindholm Emil: 23.582806752708056
12. Kremer Armin: 21.98175895827577
13. Solberg Oliver: 21.132601465326715
14. Greensmith Gus: 20.11042891863717
15. Miele Mauro: 19.324396639149548
16. Cais Erik: 17.60662379498493
17. Fourmaux Adrien: 16.81216347132177
18. Pajari Sami: 16.215266471855625
19. McErlean Josh: 16.17181553732557
20. Rossel Yohan: 14.864377310234993
21. Munster Grégoire: 14.439199482935663
22. Keferböck Johannes: 13.138754826525426
23. Boland Eamonn: 12.553631326199936
24. Turk Rok: 11.771238691249023
25. O'Brien Patrick: 11.771238691249023
26. Herczig Norbert: 11.156743462304135
27. 